In [ ]:
import openai
import requests
from flask import Flask, request, jsonify, render_template
from transformers import pipeline
from amadeus import Client, ResponseError
import pandas as pd

In [12]:
api_url = 'https://test.api.amadeus.com/v1/reference-data/locations/hotels/by-city'
params = {
    'cityCode': 'CLE',  # IATA code for Cleveland
    'radius': 5,
    'radiusUnit': 'KM',
    'ratings': '3,4,5',  # Optional: filter by star ratings
    'hotelSource': 'ALL'  # ALL, BEDBANK, or DIRECTCHAIN
}

# For authentication
auth_url = 'https://test.api.amadeus.com/v1/security/oauth2/token'
auth_data = {
    'grant_type': 'client_credentials',
    'client_id': 'RW5x5Bn2aMZYmgyHCyomxPetDteMmM2a',
    'client_secret': 'sKvaa9jS36eO7OCL'
}

# Get access token
auth_response = requests.post(auth_url, data=auth_data)
access_token = auth_response.json()['access_token']

# Use token in API request
headers = {
    'Authorization': f'Bearer {access_token}'
}

response = requests.get(api_url, headers=headers, params=params)
data = response.json()
print(data)

{'data': [{'chainCode': 'DT', 'iataCode': 'CLE', 'dupeId': 700008366, 'name': 'DBLTREE CLEVELAND DOWNTOWN', 'hotelId': 'DTCLE216', 'geoCode': {'latitude': 41.50674, 'longitude': -81.68916}, 'address': {'countryCode': 'US'}, 'distance': {'value': 0.25, 'unit': 'KM'}, 'rating': 3, 'lastUpdate': '2023-06-15T09:42:40'}, {'chainCode': 'WY', 'iataCode': 'CLE', 'dupeId': 700050873, 'name': 'WYNDHAM PLAYHOUSESQUARE HOTEL', 'hotelId': 'WYCLEPHS', 'geoCode': {'latitude': 41.5006, 'longitude': -81.68353}, 'address': {'countryCode': 'US'}, 'distance': {'value': 0.77, 'unit': 'KM'}, 'rating': 5, 'lastUpdate': '2023-06-15T09:44:48'}, {'chainCode': 'RD', 'iataCode': 'CLE', 'dupeId': 700037315, 'name': 'RADISSON CLEVELAND GATEWAY', 'hotelId': 'RDCLE627', 'geoCode': {'latitude': 41.49842, 'longitude': -81.68713}, 'address': {'countryCode': 'US'}, 'distance': {'value': 0.77, 'unit': 'KM'}, 'rating': 3, 'lastUpdate': '2023-06-15T09:43:45'}, {'chainCode': 'RZ', 'iataCode': 'CLE', 'dupeId': 700112673, 'nam

In [15]:
hotels_list = []
for hotel in data['data']:
    hotel_info = {
        'name': hotel.get('name', 'Name not available'),
        'hotel_id': hotel.get('hotelId', 'ID not available'),
        'rating': hotel.get('rating', 'Rating not available'),
    }
    hotels_list.append(hotel_info)
df = pd.DataFrame(hotels_list)
print(df)

                            name  hotel_id  rating
0     DBLTREE CLEVELAND DOWNTOWN  DTCLE216       3
1  WYNDHAM PLAYHOUSESQUARE HOTEL  WYCLEPHS       5
2     RADISSON CLEVELAND GATEWAY  RDCLE627       3
3     THE RITZ-CARLTON CLEVELAND  RZCLECLE       5
4           COMFORT INN DOWNTOWN  CICLE167       3
5    HILTON GI CLEVELAND GATEWAY  GICLE246       3
6            UNIVERSITY HOTEL AN  ILCLE101       3
7  INTERCONTINENTAL SUITES HOTEL  ICCLEICB       3
